참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import matplotlib.patches as patch # 도형 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 08.006 모어의 원<br>08.006 Mohr's circle

응력 상태로부터 주응력 등 다른 응력 상태를 그림으로 찾음<br>From a stress status, find other stress status(possibly principal stresses).

p. 357

### 2차원 응력 상태<br>2D stress status

$$
    \begin{pmatrix}
        \sigma_{x'}\\
        \tau_{x'y'}\\
    \end{pmatrix}=
    \begin{bmatrix}
        cos\theta & sin\theta \\
        -sin\theta & cos\theta\\
    \end{bmatrix}
    \begin{bmatrix}
        \sigma_{x} & \tau_{xy} \\
        \tau_{xy} & \sigma_{y}\\
    \end{bmatrix}
    \begin{pmatrix}
        cos\theta\\
        sin\theta\\
    \end{pmatrix}
$$

p. 347 eq 8.5 a), c)

$$
    \begin{cases}
        \sigma_{x'} &= \frac{\sigma_x + \sigma_y}{2} 
                    &+ \frac{\sigma_x - \sigma_y}{2}cos 2\theta  
                    &+ \tau_{xy}sin 2\theta \\
        \tau_{x'y'} &= &-\frac{\sigma_x - \sigma_y}{2}sin 2\theta 
                    &+ \tau_{xy}cos 2\theta \\
    \end{cases}
$$

행렬과 벡터를 이용하여 아래와 같이 표시할 수 있다.<br>An engineer can rewrite this equation as follows using a matrix and vectors.

$$
    \begin{pmatrix}
        \sigma_{x'}\\
        \tau_{x'y'}\\
    \end{pmatrix}
    =
    \begin{pmatrix}
        \frac{\sigma_x + \sigma_y}{2}\\
        0\\
    \end{pmatrix}
    +
    \begin{bmatrix}
        cos2\theta & sin2\theta \\
        -sin2\theta & cos2\theta\\
    \end{bmatrix}
    \begin{pmatrix}
        \frac{\sigma_x - \sigma_y}{2}\\
        \tau_{xy}\\
    \end{pmatrix}
$$

위 식의 행렬은 회전 행렬이다.<br>The matrix of equation is a rotation matrix.

따라서 그 오른쪽의 벡터를 $-2\theta$ 만큼 회전 시킬 것이다.<br>Thus it would rotate the vector on its right side by $-2\theta$.

$0 \le \theta \le \pi$ 라면, $0 \le 2\theta \le 2\pi$ 일 것이므로<br>Because $0 \le \theta \le \pi$ means $0 \le 2\theta \le 2\pi$

위 방정식은 $\begin{pmatrix} \frac{\sigma_x + \sigma_y}{2} & 0\\ \end{pmatrix} ^ T$ 을 중심으로 하는 원을 나타낸다. <br> the equation above represents a circle centered at $\begin{pmatrix} \frac{\sigma_x + \sigma_y}{2} & 0\\ \end{pmatrix} ^ T$.

#### 회전 행렬의 예<br>An example of the rotation matrix

In [ ]:
theta_deg = 30
theta_rad = np.deg2rad(theta_deg)
rot_mat = np.matrix(
    [(np.cos(2 * theta_rad), np.sin(2 * theta_rad)),
     (-np.sin(2 * theta_rad), np.cos(2 * theta_rad)),]
)
vec = np.matrix(
    [[2], 
     [1]]
)
rotated_vec = rot_mat * vec

In [ ]:
plt.clf()
ax = plt.gca()
# https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.arrow.html
plt.annotate("", xy=vec, xytext=(0, 0), arrowprops=dict(arrowstyle="->"))
plt.annotate("", xy=rotated_vec, xytext=(0, 0), arrowprops=dict(arrowstyle="->"))
ax.add_patch(plt.Circle((0, 0), radius=(2**2 + 1**2) ** 0.5, fill=None, linewidth=0.5))
plt.plot((0, vec[0, 0], rotated_vec[0, 0]),
         (0, vec[1, 0], rotated_vec[1, 0]),
         '.', alpha=0.0)
plt.text(vec[0, 0], vec[1, 0], "vec")
plt.text(rotated_vec[0, 0], rotated_vec[1, 0], "rotated_vec")
plt.axis('equal')
plt.grid(True)
plt.show()

#### 원의 방정식<br>Equation of a circle

p. 347 eq 8.5 a), c)

$$
    \begin{cases}
        \sigma_{x'} &= \frac{\sigma_x + \sigma_y}{2} 
                    &+ \frac{\sigma_x - \sigma_y}{2}cos 2\theta  
                    &+ \tau_{xy}sin 2\theta \\
        \tau_{x'y'} &= &-\frac{\sigma_x - \sigma_y}{2}sin 2\theta 
                    &+ \tau_{xy}cos 2\theta \\
    \end{cases}
$$

$$
    \begin{cases}
        \sigma_{x'} - \frac{\sigma_x + \sigma_y}{2} 
                    &= \frac{\sigma_x - \sigma_y}{2}cos 2\theta  
                    &+ \tau_{xy}sin 2\theta \\
        \tau_{x'y'} &= -\frac{\sigma_x - \sigma_y}{2}sin 2\theta 
                    &+ \tau_{xy}cos 2\theta \\
    \end{cases}
$$

두 식을 각각 제곱한 후 더하면 다음과 같다<br>Adding squares of both equation gives following.

$$
    \begin{cases}
        \left(\sigma_{x'} - \frac{\sigma_x + \sigma_y}{2} \right)^2
                    &= \left( \frac{\sigma_x - \sigma_y}{2}cos 2\theta + \tau_{xy}sin 2\theta \right)^2 \\
        \tau_{x'y'}^2 &= \left( -\frac{\sigma_x - \sigma_y}{2}sin 2\theta 
                    + \tau_{xy}cos 2\theta \right)^2 \\
    \end{cases}
$$

$$
    \begin{cases}
        \left(\sigma_{x'} - \frac{\sigma_x + \sigma_y}{2} \right)^2
                    &= \left( \frac{\sigma_x - \sigma_y}{2} \right)^2 cos^2 2\theta 
                    + 2 \frac{\sigma_x - \sigma_y}{2}cos 2\theta \tau_{xy}sin 2\theta
                    + \tau_{xy}^2 sin^2 2\theta \\
        \tau_{x'y'}^2 &= \left( \frac{\sigma_x - \sigma_y}{2} \right)^2 sin^2 2\theta 
                    - 2 \frac{\sigma_x - \sigma_y}{2}sin 2\theta \tau_{xy}cos 2\theta
                    + \tau_{xy}^2 cos^2 2\theta \\
    \end{cases}
$$

$$
\begin{align}
    \left(\sigma_{x'} - \frac{\sigma_x + \sigma_y}{2} \right)^2 
    +\tau_{x'y'}^2 = \left( \frac{\sigma_x - \sigma_y}{2} \right)^2 \left( cos^2 2\theta + sin^2 2\theta \right) + \tau_{xy}^2 \left( sin^2 2\theta + cos^2 2\theta \right)^2\\
\end{align}
$$

$$
\begin{align}
    \left(\sigma_{x'} - \frac{\sigma_x + \sigma_y}{2} \right)^2 
    +\tau_{x'y'}^2 = \left( \frac{\sigma_x - \sigma_y}{2} \right)^2 + \tau_{xy}^2\\
\end{align}
$$

위 식을 다음 원의 방정식과 비교해 보자<br>Let's compare this equation with a circle's equation.

$$
\begin{align}
    \left(x - x_c \right)^2 +y^2 = R^2\\
\end{align}
$$

### 모어의 원<br>Mohr's circle

아래 값은 예제 8.4로 부터 가져온 것이다.<br>Following values are from example 8.4

In [ ]:
sigma_x_Pa = 30e6
sigma_y_Pa = 60e6
tau_xy_Pa = 40e6

$\sigma_{x'}$ 와 $\sigma_{y'}$ 를 $x$축 위에 표시한다.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, sigma_y_Pa), (0, 0), '.')
plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')

In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

두 점의 중간점을 중심으로 표시한다.<br>Indicate the mid-point as the center of the circle.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_y_Pa), (0, 0, 0), '.')
plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')
plt.text((sigma_x_Pa + sigma_y_Pa) * 0.5, 0, '$\\frac{1}{2}\\left(\\sigma_x+\\sigma_y\\right)$')

In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

$\sigma_y$로 부터 $\tau_{xy}$ 길이의 수직선을 위로 그린다.<br>Starting from $\sigma_y$, draw an upward vertical line $\tau_{xy}$ long.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_y_Pa), (0, 0, 0), '.')
plt.plot((sigma_y_Pa, sigma_y_Pa, ), (0, tau_xy_Pa), '.-')

plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')
plt.text((sigma_x_Pa + sigma_y_Pa) * 0.5, 0, '$\\frac{1}{2}\\left(\\sigma_x+\\sigma_y\\right)$')
plt.text(sigma_y_Pa, 0.5 * tau_xy_Pa, '$\\tau_{xy}$')


In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

수직선 위 끝으로 부터 원의 중심 까지 선분을 그린다.<br>From the top of the vertical line, draw a segment to the center of the circle.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_y_Pa), (0, 0, 0), '.')
plt.plot((sigma_y_Pa, sigma_y_Pa,), (0, tau_xy_Pa), '.-')
plt.plot((sigma_y_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5,), (tau_xy_Pa, 0), '.-')

plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')
plt.text((sigma_x_Pa + sigma_y_Pa) * 0.5, 0, '$\\frac{1}{2}\\left(\\sigma_x+\\sigma_y\\right)$')
plt.text(sigma_y_Pa, 0.5 * tau_xy_Pa, '$\\tau_{xy}$')


In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

같은 선분을 원의 중심으로 부터 그린다.<br>Draw the same segment from the center.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_y_Pa), (0, 0, 0), '.')
plt.plot((sigma_y_Pa, sigma_y_Pa,), (0, tau_xy_Pa), '.-')
plt.plot((sigma_y_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5,), (tau_xy_Pa, 0), '.-')
plt.plot(((sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_x_Pa), (0, -tau_xy_Pa,), '.-')

plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')
plt.text((sigma_x_Pa + sigma_y_Pa) * 0.5, 0, '$\\frac{1}{2}\\left(\\sigma_x+\\sigma_y\\right)$')
plt.text(sigma_y_Pa, 0.5 * tau_xy_Pa, '$\\tau_{xy}$')


In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

$\sigma_x$ 로 부터  $\tau_{xy}$ 길이의 수직선을 아래로 그리면 이 선분과 만난다.<br>A downward vertical segment from $\sigma_x$ of length $\tau_{xy}$ will meet with this segment.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_y_Pa), (0, 0, 0), '.')
plt.plot((sigma_y_Pa, sigma_y_Pa,), (0, tau_xy_Pa), '.-')
plt.plot((sigma_y_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5,), (tau_xy_Pa, 0), '.-')
plt.plot(((sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_x_Pa), (0, -tau_xy_Pa,), '.-')
plt.plot((sigma_x_Pa, sigma_x_Pa), (0, -tau_xy_Pa,), '.-')

plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')
plt.text((sigma_x_Pa + sigma_y_Pa) * 0.5, 0, '$\\frac{1}{2}\\left(\\sigma_x+\\sigma_y\\right)$')
plt.text(sigma_y_Pa, 0.5 * tau_xy_Pa, '$\\tau_{xy}$')
plt.text(sigma_x_Pa, -0.5 * tau_xy_Pa, '$\\tau_{xy}$', ha='right')


In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

중점을 중심으로 선분을 회전시키면 그 끝점은 원을 그린다.<br>If an engineer rotates the segment around the midpoint, its other end point would make a circle.

In [ ]:
plt.clf()
plt.plot((sigma_x_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_y_Pa), (0, 0, 0), '.')
plt.plot((sigma_y_Pa, sigma_y_Pa,), (0, tau_xy_Pa), '.-')
plt.plot((sigma_y_Pa, (sigma_x_Pa + sigma_y_Pa) * 0.5,), (tau_xy_Pa, 0), '.-')
plt.plot(((sigma_x_Pa + sigma_y_Pa) * 0.5, sigma_x_Pa), (0, -tau_xy_Pa,), '.-')
plt.plot((sigma_x_Pa, sigma_x_Pa), (0, -tau_xy_Pa,), '.-')
plt.gca().add_patch(
    plt.Circle(((sigma_x_Pa + sigma_y_Pa) * 0.5, 0), 
               radius=(((sigma_x_Pa - sigma_y_Pa) * 0.5)**2 + tau_xy_Pa**2) ** 0.5, 
               fill=None, linewidth=0.5)
)

plt.text(sigma_x_Pa, 0, '$\\sigma_x$')
plt.text(sigma_y_Pa, 0, '$\\sigma_y$')
plt.text((sigma_x_Pa + sigma_y_Pa) * 0.5, 0, '$\\frac{1}{2}\\left(\\sigma_x+\\sigma_y\\right)$')
plt.text(sigma_y_Pa, 0.5 * tau_xy_Pa, '$\\tau_{xy}$')
plt.text(sigma_x_Pa, -0.5 * tau_xy_Pa, '$\\tau_{xy}$', ha='right')


In [ ]:
plt.plot(0, 0, alpha=0)
plt.xlabel('$\\sigma$')
plt.ylabel('$\\tau$')
plt.grid(True)
plt.axis('equal')
plt.show()

또는 아래와 같이 같이 표시할 수도 있을 것이다.<br>Also, an engineer can represent as follows.

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

In [ ]:
ax0 = plt.subplot(1, 2, 1)
draw_diagrams.draw_stress_2d(sigma_x_Pa, 
                             sigma_y_Pa, 
                             tau_xy_Pa, ax=ax0)
ax1 = plt.subplot(1, 2, 2)
draw_diagrams.plot_mohr_circle(sigma_x_Pa, sigma_y_Pa, tau_xy_Pa)
plt.show()